# Import libraries
**pandas** - data manipulation and analysis (i.e. DataFrame, integrated indexing).<br>
**numpy** - multi-dimensional array manipulation.<br>
**sklearn** - machine learning library with various classification, regression and clustering algorithms <br>
**plotly** - graphing library that supports interactive graphs <br>
**logger** - custom logger wrapper built on top of Python logger for event logging <br>
**itertools** - fast & memory-efficient looping tool

In [1]:
import pandas as pd
import numpy as np
import hashlib
import os 
from utils import logger
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectFromModel
from sklearn import datasets

from sklearn.linear_model import LassoCV
from sklearn.linear_model import Lasso
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
from utils import logger
import sys

import plotly.plotly as py
import plotly.graph_objs as go
import itertools as it

from sklearn.metrics import classification_report,confusion_matrix
from sklearn.metrics import roc_curve, auc

# Feature Selection & Extraction Helpers
**Purpose** - reduces overfitting, improves accuracy (sometimes), reduces training time.
## Feature Selection 
**In a nutshell**: runs analysis on dataset and returns the best set of features which can be used for classification. <br>
* **Filter Method** - calculate correlation of feature variable with target variable. The features are ranked by the score and either selected to be kept or removed from the dataset. (eg. Chi-square, information gain, correlation coefficient scores)<br>
* **Wrapper Method** - search for combination of variables that performs the best using a certain heuristic (e.g. subset, forward, backward elimination) <br>
* **Embedded method** - learn which features best contribute to accuracy while model is being created using regularization methods (e.g. Lasso, Ridge, Elastic Net)<br>

## Feature Extraction 
**In a nutshell**: use initial features to build derived features that are more informative and non-redundent than the original dataset. This is different from feature selection in that feature extraction combines all feature information into newly-created features without completely eliminating low-contributing features (e.g. PCA, NMF, kernel PCA, Graph-based kernel PCA etc.). <br>

## What we are using:
* **Lasso** - a **feature selection** method that uses cross-validation LASSO regression to filter out un-important features <br>
* **PCA** - a **feature extraction** method uses the correlation between some dimensions and tries to provide a minimum number of variables that keeps the maximum amount of variation or information about how the original data is distributed. <br>
* **t-SNE** - a **dimensionality reduction** method that maps multi-dimensional data to a lower dimensional space (typcially 2 or 3), by constructing pairwise probability distribution, modeling similar objects then minimizing KL divergence between original data and lower-dimensional data. This is mainly used for visualizing high-dimensional data, not for feeding directly into models. <br>

In [2]:
def lassoSelection(X_train, y_train, n):
    '''
    Lasso feature selection.  Select n features. 
    '''
    #lasso feature selection
    #print (X_train)
    clf = LassoCV(max_iter=10000,tol=0.001)
    sfm = SelectFromModel(clf,threshold=0)
    sfm.fit(X_train, y_train)
    X_transform = sfm.transform(X_train)
    n_features = X_transform.shape[1]

    # 	print("n_features=",n_features)
    #print(n_features)
    while n_features > n:
        sfm.threshold += 0.01
        X_transform = sfm.transform(X_train)
        n_features = X_transform.shape[1]
        print ("n_features =",n_features)
    features = [index for index,value in enumerate(sfm.get_support()) if value == True  ]
    logger.info("selected features are {}".format(features))
    logger.info("Features selected from Lasso using SelectFromModel with threshold {:6.4f}".format(sfm.threshold))
    return features

In [3]:
def pcaSelection(X_train, X_test, n):
    '''
    PCA feature selection.  Select n features. 
    '''
    pca = PCA(n_components=n)
    pca.fit(X_train)
    X_train_new = pca.transform(X_train)
    X_test_new = pca.transform(X_test)
    logger.info("X_train size after PCA: {}".format(X_train_new.shape))
    logger.info("X_test size after PCA: {}".format(X_test_new.shape))
    logger.info("Cumulative explained variation for {} principal components: {:6.4f}".format(n,np.sum(pca.explained_variance_ratio_)))
    return [X_train_new,X_test_new]

In [4]:
def tsneSelection(X_train, n, v):
    '''
    t-distributed Stochastic Neighbor Embedding feature selection.  Select n features.
    Input:
    X_train - dataset with dimension [P-samples x Q-features]
    n - number of desired features after feature reduction
    v - 1 for verbose, 0 for slience
    Output:
    X_train_new - dataset with dimension [P-samples x n-features] 
    '''
    tsne = TSNE(n_components=n,verbose=v)
    X_train_new = tsne.fit_transform(X_train)
    logger.info("X_train size after tSNE: {}".format(X_train_new.shape))
    return X_train_new

# Scatter Plot Helpers
**2D Scatter Plot** - graph multi-class data with 2 features in a 2D plot<br>
**3D Scatter Plot** - graph multi-class data with 3 features in a 3D plot<br>

In [17]:
# Scatter plot 2D & 3D
num_class = 36
colors = it.cycle(["aquamarine", "crimson", "darkseagreen", "deeppink","wheat","violet","fuchsia","turquoise",\
                   "ivory", "honeydew", "rosybrown","red","lemonchiffon","darkorchid","mintcream","papayawhip",\
                   "beige","darkcyan","firebrick","deepskyblue","seashell","mediumpurple","goldenrod","lightcoral",\
                   "limegreen","cadetblue","darkmagenta","ghostwhite","gainsboro","paleturquoise","teal","peru",\
                  "maroon","olivedrab","springgreen","yellowgreen"])
classes = it.cycle(['Normal','Breast', 'Uterine Corpus', 'Head', 'Kidney Renal Clear', 'Lung Adenocarcinoma', 'Brain', 'Thyroid', 'Prostate', 'Ovarian', 'Lung Squamous', 'Skin', 'Colon', 'Stomach', 'Bladder', 'Liver', 'Cervical', 'Kidney Renal Papillary', 'Leukemia', 'Sarcoma', 'Esophageal', 'Pheochromocytoma', 'Pancreatic', 'Rectum', 'Testicular', 'Wilms', 'Thymoma', 'Mesothelioma', 'Adrenocortical', 'Uveal', 'Kidney Chromophobe', 'Uterine Carcinosarcoma', 'Lymphoid', 'Rhabdoid', 'Cholangiocarcinoma'])

classes_labels = ['Normal','Breast', 'Uterine Corpus', 'Head', 'Kidney Renal Clear', 'Lung Adenocarcinoma', 'Brain', 'Thyroid', 'Prostate', 'Ovarian', 'Lung Squamous', 'Skin', 'Colon', 'Stomach', 'Bladder', 'Liver', 'Cervical', 'Kidney Renal Papillary', 'Leukemia', 'Sarcoma', 'Esophageal', 'Pheochromocytoma', 'Pancreatic', 'Rectum', 'Testicular', 'Wilms', 'Thymoma', 'Mesothelioma', 'Adrenocortical', 'Uveal', 'Kidney Chromophobe', 'Uterine Carcinosarcoma', 'Lymphoid', 'Rhabdoid', 'Cholangiocarcinoma']

def scatter2D(X_train_2d):
    '''
    Function to genrate traces for 2D scatter plot
    Args: 2-feature X_train of dimension [?,2]
    Return: list of scatter plot trace objects
    '''
    data=[]
    for label in range(0,num_class):
        filtered_idx = np.argwhere(y_train==label)[:,0]
        trace = go.Scatter(
            x=X_train_2d[filtered_idx,0],
            y=X_train_2d[filtered_idx,1],
            mode='markers',
            marker=dict(
                size=5,
                line=dict(
                    color=next(colors),
                    width=0.1
                    ),
                opacity=0.5
                ),
            name=next(classes)
            )
        data.append(trace)
    return data


def scatter3D(X_train_3d):
    '''
    Function to generate traces for 3D scatter plot
    Args: 3-feature X_train of dimension [?,3]
    ReturnL list of scatter plot trace objects
    '''
    data=[]
    for label in range(0,num_class):
        filtered_idx = np.argwhere(y_train==label)[:,0]
        trace = go.Scatter3d(
            x=X_train_3d[filtered_idx,0],
            y=X_train_3d[filtered_idx,1],
            z=X_train_3d[filtered_idx,2],
            mode='markers',
            marker=dict(
                size=5,
                line=dict(
                    color=next(colors),
                    width=0.1
                    ),
                opacity=0.5
                ),
            name=next(classes)
            )
        data.append(trace)
    return data

# Model Helper

## Estimators
### Linear Estimators
* **Logistic Regresion** - <br>

### Ensemble Estimators
**In a nutshell** - weighted combinations of simple predictors(e.g. one-level decision trees), correct predictor is given more weight. Boosting algorithms focuses new learners on sample points that previous predictors get wrong. <br>

* **Random Forest Classifier** - divides data into sub-samples and construct a binary decision trees for each set of sub-samples. Gini optimizer is used to find the best "split" for each decision junction. Validation or test data is then pass through the decision tree, which outputs probablity for each class. These probabilities of each tree is then averaged for a final classification decision. <br>
* **Extra Trees Classifier** - similar to Random Forest, but a random value is selected for each split <br>
* **AdaBoost Classifier** - Uses N number of 1-level decision tree (optimized by Gini Impurity cost function) as base classifier. Wrongly-predicted samples are given larger weights for the next round of decsion tree optimization. Incorrect classifiers' contribution is shrinked by learning_rate. At the end, the final classifer is a weighted combination of all n estimators.
 * By setting high number of estimaters, and low learning rate, the model will converge with high accuracy at the expense of long computation time. <br>
* **Gradient Boosting Classifier** - use simple regression estimator to fit data, use error residual(MSE) to iteratively fit training data. Simple regressors are added up in a stage-wise fashion to get a complex regression model. <br>

### SVN Estimator
* **SVC** - <br>

### Neural Network Estimator
* **Multi-Layer Perceptron (MLP) Classifier** - <br>

## K-fold Hyperparameter selection
**GridSearchCV** - exhaustively predicts and scores all parameter combinations for an estimator

In [6]:
def model_fit_predict(X_train,X_test,y_train,y_test,v):

    # np.random.seed(2018)
    from sklearn.linear_model import LogisticRegression
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.ensemble import AdaBoostClassifier
    from sklearn.tree import DecisionTreeClassifier
    from sklearn.ensemble import GradientBoostingClassifier
    from sklearn.ensemble import ExtraTreesClassifier
    from sklearn.svm import SVC
    from sklearn.neural_network import MLPClassifier
    from sklearn.metrics import precision_score
    from sklearn.metrics import accuracy_score
    from sklearn.metrics import f1_score
    from sklearn.metrics import recall_score
    from sklearn.preprocessing import label_binarize
    models = {
        'LogisticRegression': LogisticRegression(random_state=0, multi_class='ovr',solver='lbfgs',max_iter=10000,tol=0.001,verbose=v),
        'ExtraTreesClassifier': ExtraTreesClassifier(random_state=0, verbose=v),
        'RandomForestClassifier': RandomForestClassifier(random_state=0, verbose=v),
#         'AdaBoostClassifier': AdaBoostClassifier(DecisionTreeClassifier(max_depth=1),random_state=0),
#         'GradientBoostingClassifier': GradientBoostingClassifier(n_estimators = 5000, random_state=0, n_iter_no_change=10, verbose=v),
        'SVC': SVC(random_state=0, decision_function_shape='ovo',max_iter=10000,tol=0.001,verbose=v),
        'MLP':MLPClassifier(random_state=0, hidden_layer_sizes=(500,100,50),solver='adam',max_iter=1000,verbose=v,\
                            learning_rate ='adaptive',activation='relu')
    }
    tuned_parameters = {
        'LogisticRegression':{'C': [0.25,0.5,1]},
        'ExtraTreesClassifier': { 'n_estimators': [200,500,700,1000] },
        'RandomForestClassifier': { 'n_estimators': [200,500,1000,5000],'min_samples_leaf': [1,2,3]},
#         'AdaBoostClassifier': { 'n_estimators': [200,500,1000,5000],'learning_rate': [0.05,0.2, 0.7]},
#         'GradientBoostingClassifier': { 'learning_rate': [0.05,0.2, 0.7] },
        'SVC': { 'kernel': ['rbf'], 'C': [1, 10], 'gamma': [0.001, 0.0001] },
        'MLP':{ 'batch_size':[50,200],'alpha':[0.0001,0.005,0.01],'tol':[0.01,0.001,0.0001]}
    }
    accuracies= {}
    confusion_mat = {}
    report = {}
    fpr_all = {}
    tpr_all ={}
    for key in models:
        print("Running",key,"...")
        clf = GridSearchCV(models[key], tuned_parameters[key], scoring=None,  refit=True, cv=2, verbose=v)
        clf.fit(X_train,y_train)
        print(clf.best_params_)
        # Compute Metrics
        y_test_predict = clf.predict(X_test)
        accuracy = accuracy_score(y_test, y_test_predict) #subset accuracy 
        accuracies[key] = accuracy
        confusion_mat[key]=confusion_matrix(y_test,y_test_predict)
        report[key]= classification_report(y_test,y_test_predict,target_names=classes_labels)
        # Compute ROC curve and ROC area for each class
        fpr = dict()
        tpr = dict()
        y_test_bin = label_binarize(y_test, classes=[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35])
        y_test_predict_bin = label_binarize(y_test_predict, classes=[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35])  
        for i in range(0,num_class-1):
            fpr[i], tpr[i], _ = roc_curve(y_test_bin[:,i], y_test_predict_bin[:,i])
        fpr["micro"], tpr["micro"], _ = roc_curve(y_test_bin.ravel(), y_test_predict_bin.ravel())
        fpr_all[key]=fpr
        tpr_all[key]=tpr
        print(accuracies[key])
        print(confusion_mat[key])
        print(report[key])
    return accuracies, confusion_mat, report, fpr_all, tpr_all

# Pre-process Data
1. **Separate X (features) and y (labels)** <br>
2. **Split training (70%) and testing (30%) dataset** <br>
3. **Standardize data** - scale features such that they are:
  1. zero-mean
  2. one-variance

In [7]:
data_file = "../data/miRNA.csv" # directory to miRNA_matrix.csv

# Get dataset from csv
df = pd.read_csv(data_file)
y_data = df.pop('label').values
df.pop('file_id')
columns =df.columns
X_data = df.values
num_features_orig = X_data.shape[1]
logger.info("Original dataset size: {}".format(X_data.shape[0]))
logger.info("Total feature num: {}".format(num_features_orig))

# split the data to train and test set
X_train, X_test, y_train, y_test = train_test_split(X_data, y_data, test_size=0.3, random_state=0)


logger.info("Training dataset size: {}".format(X_train.shape[0]))
logger.info("Testing dataset size: {}".format(X_test.shape[0]))
# # standardize the data (zero-mean,uniform variance)
scaler = StandardScaler().fit(X_train.astype(np.float64))
X_train = scaler.transform(X_train.astype(np.float64))
X_test = scaler.transform(X_test.astype(np.float64))
logger.info("Mean of X-data features before standardization: {:6.4f}".format(sum(X_data.mean(axis=0))/num_features_orig))
logger.info("STD of X-data features before standardization: {:6.4f}".format(sum(X_data.std(axis=0))/num_features_orig))
logger.info("Mean of X-train features after standardization: {:6.4f}".format(sum(X_train.mean(axis=0))/num_features_orig))
logger.info("STD of X-train features after standardization: {:6.4f}".format(sum(X_train.std(axis=0))/num_features_orig))
logger.info("Mean of X-test features after standardization: {:6.4f}".format(sum(X_test.mean(axis=0))/num_features_orig))
logger.info("STD of X-test features after standardization: {:6.4f}".format(sum(X_test.std(axis=0))/num_features_orig))

# Define Graph layout
layout = go.Layout(
    margin=dict(
        l=0,
        r=0,
        b=0,
        t=0
        )
)

[2018-10-22 17:42:10,616 - GDC - INFO] Original dataset size: 11486
[2018-10-22 17:42:10,619 - GDC - INFO] Total feature num: 1881
[2018-10-22 17:42:11,902 - GDC - INFO] Training dataset size: 8040
[2018-10-22 17:42:11,903 - GDC - INFO] Testing dataset size: 3446
[2018-10-22 17:42:12,893 - GDC - INFO] Mean of X-data features before standardization: 2458.7368
[2018-10-22 17:42:13,228 - GDC - INFO] STD of X-data features before standardization: 4450.1113
[2018-10-22 17:42:13,250 - GDC - INFO] Mean of X-train features after standardization: -0.0000
[2018-10-22 17:42:13,400 - GDC - INFO] STD of X-train features after standardization: 0.9351
[2018-10-22 17:42:13,409 - GDC - INFO] Mean of X-test features after standardization: -0.0069
[2018-10-22 17:42:13,461 - GDC - INFO] STD of X-test features after standardization: 0.9125


# Fit Model after LASSO-Selected Features

In [ ]:
# LASSO feature selection
n = 50
feaures_columns = lassoSelection(X_train, y_train, n)
# feaures_columns = [25, 92, 119, 163, 166, 168, 181, 187, 194, 216, 240, 241, 248, \
# 253, 271, 272, 273, 282, 285, 287, 295, 305, 306, 336, 337, 339, 341, 351, 352, 488, \
# 495, 503, 511, 544, 588, 593, 641, 764, 1063, 1090, 1100, 1126, 1395, 1461, 1509, 1523, 1834, 1848, 1872]
scores_lasso, mat_lasso, report_lasso, fpr_lasso, tpr_lasso = model_fit_predict(X_train[:,feaures_columns],X_test[:,feaures_columns],y_train,y_test,1)

# Fit Model with PCA-Reduced Features

In [8]:
# PCA feature reduction to n-components
n = 50 # 650 for ~80% representation
X_train_pca, X_test_pca = pcaSelection(X_train, X_test, n)
scores_pca, mat_pca, report_pca, fpr_pca, tpr_pca = model_fit_predict(X_train_pca,X_test_pca,y_train,y_test,1)

[2018-10-22 17:42:15,905 - GDC - INFO] X_train size after PCA: (8040, 50)
[2018-10-22 17:42:15,906 - GDC - INFO] X_test size after PCA: (3446, 50)
[2018-10-22 17:42:15,908 - GDC - INFO] Cumulative explained variation for 50 principal components: 0.3773
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=-1)]: Using backend SequentialBackend with 1 concurrent workers.


Running LogisticRegression ...
Fitting 2 folds for each of 3 candidates, totalling 6 fits


[Parallel(n_jobs=-1)]: Done  35 out of  35 | elapsed:    3.1s finished
[Parallel(n_jobs=-1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=-1)]: Done  35 out of  35 | elapsed:    3.1s finished
[Parallel(n_jobs=-1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=-1)]: Done  35 out of  35 | elapsed:    3.4s finished
[Parallel(n_jobs=-1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=-1)]: Done  35 out of  35 | elapsed:    3.2s finished
[Parallel(n_jobs=-1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=-1)]: Done  35 out of  35 | elapsed:    4.1s finished
[Parallel(n_jobs=-1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=-1)]: Done  35 out of  35 | elapsed:    4.0s finished
[Parallel(n_jobs=1)]: Done   6 out of   6 | elapsed:   21.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_j

{'C': 0.25}
0.8537434706906558
[[155   3   1 ...   0   0   0]
 [  1 302   6 ...   0   0   0]
 [  2   2 156 ...   0   0   0]
 ...
 [  0   0   0 ...  13   0   0]
 [  0   1   0 ...   0  21   0]
 [  2   0   1 ...   0   0   3]]
                        precision    recall  f1-score   support

                Normal       0.84      0.77      0.81       201
                Breast       0.90      0.93      0.91       324
        Uterine Corpus       0.83      0.90      0.86       174
                  Head       0.70      0.78      0.74       152
    Kidney Renal Clear       0.93      0.97      0.95       155
   Lung Adenocarcinoma       0.81      0.81      0.81       162
                 Brain       0.98      1.00      0.99       170
               Thyroid       0.99      0.99      0.99       149
              Prostate       0.96      0.99      0.98       155
               Ovarian       0.99      0.95      0.97       146
         Lung Squamous       0.67      0.73      0.70       142
        

[Parallel(n_jobs=-1)]: Done 200 out of 200 | elapsed:    2.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 200 out of 200 | elapsed:    0.4s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 200 out of 200 | elapsed:    0.3s finished
[Parallel(n_jobs=-1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=-1)]: Done 200 out of 200 | elapsed:    1.9s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 200 out of 200 | elapsed:    0.5s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 200 out of 200 | elapsed:    0.4s finished
[Parallel(n_jobs=-1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=-1)]: Done 500 out of 500 | elapsed:    4.9s finished
[Parallel(n_jobs=1)

...

Iteration 1, loss = 1.89098807
Iteration 2, loss = 0.76122228
Iteration 3, loss = 0.56984846
Iteration 4, loss = 0.47515434
Iteration 5, loss = 0.39668395
Iteration 6, loss = 0.36234601
Iteration 7, loss = 0.32997937
Iteration 8, loss = 0.30130141
Iteration 9, loss = 0.29380155
Iteration 10, loss = 0.25772865
Iteration 11, loss = 0.25536134
Iteration 12, loss = 0.22854882
Iteration 13, loss = 0.21306177
Iteration 14, loss = 0.20974409
Iteration 15, loss = 0.19125475
Iteration 16, loss = 0.17268775
Iteration 17, loss = 0.16281001
Iteration 18, loss = 0.15583288
Iteration 19, loss = 0.14764876
Iteration 20, loss = 0.14807116
Iteration 21, loss = 0.13653641
Iteration 22, loss = 0.12369356
Iteration 23, loss = 0.12752706
Iteration 24, loss = 0.12915897
Iteration 25, loss = 0.11393754
Iteration 26, loss = 0.11683530
Iteration 27, loss = 0.15026716
Iteration 28, loss = 0.28271499
Iteration 29, loss = 0.22529327
Iteration 30, loss = 0.16633758
Iteration 31, loss = 0.12997003
Iteration 32, los

Iteration 53, loss = 0.08543172
Training loss did not improve more than tol=0.000100 for 10 consecutive epochs. Stopping.
Iteration 1, loss = 2.83909818
Iteration 2, loss = 1.60321089
Iteration 3, loss = 0.98233892
Iteration 4, loss = 0.73763850
Iteration 5, loss = 0.60018505
Iteration 6, loss = 0.54208009
Iteration 7, loss = 0.48515687
Iteration 8, loss = 0.48145821
Iteration 9, loss = 0.43628512
Iteration 10, loss = 0.39372728
Iteration 11, loss = 0.36061046
Iteration 12, loss = 0.32217376
Iteration 13, loss = 0.32439597
Iteration 14, loss = 0.28675891
Iteration 15, loss = 0.26781075
Iteration 16, loss = 0.24347012
Iteration 17, loss = 0.27653717
Iteration 18, loss = 0.23077510
Iteration 19, loss = 0.22013656
Iteration 20, loss = 0.24190666
Iteration 21, loss = 0.22499126
Iteration 22, loss = 0.21525032
Iteration 23, loss = 0.24843768
Iteration 24, loss = 0.19418223
Iteration 25, loss = 0.17952533
Iteration 26, loss = 0.20285789
Iteration 27, loss = 0.16846199
Iteration 28, loss = 0.

Iteration 53, loss = 0.04781345
Iteration 54, loss = 0.04213783
Iteration 55, loss = 0.03965659
Iteration 56, loss = 0.03827247
Iteration 57, loss = 0.04243191
Iteration 58, loss = 0.04019068
Iteration 59, loss = 0.03482398
Iteration 60, loss = 0.03443078
Iteration 61, loss = 0.03217319
Iteration 62, loss = 0.03133152
Iteration 63, loss = 0.03267896
Iteration 64, loss = 0.03490120
Iteration 65, loss = 0.03230603
Iteration 66, loss = 0.03105132
Iteration 67, loss = 0.03189131
Iteration 68, loss = 0.03025586
Iteration 69, loss = 0.02916870
Iteration 70, loss = 0.02880297
Iteration 71, loss = 0.03228844
Iteration 72, loss = 0.03400321
Iteration 73, loss = 0.03733685
Iteration 74, loss = 0.06152652
Iteration 75, loss = 0.10469629
Iteration 76, loss = 0.15118244
Iteration 77, loss = 0.14258243
Iteration 78, loss = 0.15987755
Iteration 79, loss = 0.09573273
Iteration 80, loss = 0.09470215
Training loss did not improve more than tol=0.001000 for 10 consecutive epochs. Stopping.
Iteration 1, l

[Parallel(n_jobs=1)]: Done  36 out of  36 | elapsed: 12.4min finished


Iteration 1, loss = 1.39410269
Iteration 2, loss = 0.60986172
Iteration 3, loss = 0.47039454
Iteration 4, loss = 0.40156038
Iteration 5, loss = 0.35958613
Iteration 6, loss = 0.33102991
Iteration 7, loss = 0.32649148
Iteration 8, loss = 0.28886155
Iteration 9, loss = 0.25687240
Iteration 10, loss = 0.24187161
Iteration 11, loss = 0.22393080
Iteration 12, loss = 0.21840669
Iteration 13, loss = 0.20643037
Iteration 14, loss = 0.21518579
Iteration 15, loss = 0.19802256
Iteration 16, loss = 0.17105646
Iteration 17, loss = 0.17530611
Iteration 18, loss = 0.15748044
Iteration 19, loss = 0.14434749
Iteration 20, loss = 0.13322664
Iteration 21, loss = 0.13164214
Iteration 22, loss = 0.15572702
Iteration 23, loss = 0.14702569
Iteration 24, loss = 0.12757780
Iteration 25, loss = 0.12609107
Iteration 26, loss = 0.12514192
Iteration 27, loss = 0.10473835
Iteration 28, loss = 0.10747030
Iteration 29, loss = 0.09511333
Iteration 30, loss = 0.08732556
Iteration 31, loss = 0.08617562
Iteration 32, los

In [18]:
from sklearn.metrics import auc
models = {'LogisticRegression','ExtraTreesClassifier','RandomForestClassifier','AdaBoostClassifier',\
          'GradientBoostingClassifier','SVC','MLP'}
def ROCplot(fpr,tpr):
    '''
    Function to genrate traces for 2D scatter plot
    Args: 2-feature X_train of dimension [?,2]
    Return: list of scatter plot trace objects
    '''
    data=[]
    lw = 1
    trace_mid = go.Scatter(x=[0, 1], y=[0, 1], 
                        mode='lines', 
                        line=dict(color='navy', width=lw, dash='dash'),
                        name='Random Classification')
    data.append(trace_mid)
    for label in range (0,num_class-1):
        trace = go.Scatter(x=fpr[label], y=tpr[label], 
                            mode='lines', 
                            line=dict(color=next(colors), width=lw),
                            name= next(classes) + ' (area = %0.2f)' % auc(fpr[label], tpr[label])
                           )
        data.append(trace)
    trace_avg = go.Scatter(x=fpr['micro'], y=tpr['micro'], 
                        mode='lines', 
                        line=dict(color=next(colors), width=lw),
                        name= 'micro_avg' + ' (area = %0.2f)' % auc(fpr['micro'], tpr['micro'])
                       )
    data.append(trace_avg)
    return data


logistic_roc_data = ROCplot(fpr_pca['LogisticRegression'],tpr_pca['LogisticRegression'])
layout = go.Layout(title="Logistic Regression ROC curve",
                   xaxis=dict(title='False Positive Rate'),
                   yaxis=dict(title='True Positive Rate'))
fig_roc_1 = go.Figure(data=logistic_roc_data[:-1], layout=layout)
py.iplot(fig_roc_1)


In [21]:
extratrees_roc_data = ROCplot(fpr_pca['ExtraTreesClassifier'],tpr_pca['ExtraTreesClassifier'])
layout = go.Layout(title="Extra Trees Classifier ROC curve",
                   xaxis=dict(title='False Positive Rate'),
                   yaxis=dict(title='True Positive Rate'))
fig_roc_2 = go.Figure(data=extratrees_roc_data[:-1], layout=layout)
py.iplot(fig_roc_2)

In [22]:
rf_roc_data = ROCplot(fpr_pca['RandomForestClassifier'],tpr_pca['RandomForestClassifier'])
layout = go.Layout(title="Random Forest Classifier ROC curve",
                   xaxis=dict(title='False Positive Rate'),
                   yaxis=dict(title='True Positive Rate'))
fig_roc_3 = go.Figure(data=rf_roc_data[:-1], layout=layout)
py.iplot(fig_roc_3)

In [ ]:
adaboost_roc_data = ROCplot(fpr_pca['AdaBoostClassifier'],tpr_pca['AdaBoostClassifier'])
layout = go.Layout(title="Adaboost Classifier ROC curve",
                   xaxis=dict(title='False Positive Rate'),
                   yaxis=dict(title='True Positive Rate'))
fig_roc_4 = go.Figure(data=adaboost_roc_data, layout=layout)
py.iplot(fig_roc_4)

In [ ]:
gd_roc_data = ROCplot(fpr_pca['GradientBoostingClassifier'],tpr_pca['GradientBoostingClassifier'])
layout = go.Layout(title="Gradient Boosting Classifier ROC curve",
                   xaxis=dict(title='False Positive Rate'),
                   yaxis=dict(title='True Positive Rate'))
fig_roc_5 = go.Figure(data=gd_roc_data, layout=layout)
py.iplot(fig_roc_5)

In [24]:
svc_roc_data = ROCplot(fpr_pca['SVC'],tpr_pca['SVC'])
layout = go.Layout(title="SVC Classifier ROC curve",
                   xaxis=dict(title='False Positive Rate'),
                   yaxis=dict(title='True Positive Rate'))
fig_roc_6 = go.Figure(data=svc_roc_data[:-1], layout=layout)
py.iplot(fig_roc_6)

In [25]:
mlp_roc_data = ROCplot(fpr_pca['MLP'],tpr_pca['MLP'])
layout = go.Layout(title="Multi-Layer Perceptron Classifier ROC curve",
                   xaxis=dict(title='False Positive Rate'),
                   yaxis=dict(title='True Positive Rate'))
fig_roc_7 = go.Figure(data=mlp_roc_data[:-1], layout=layout)
py.iplot(fig_roc_7)

In [98]:
# 5 estimators

lw = 2
trace_mid = go.Scatter(x=[0, 1], y=[0, 1],
                    mode='lines', 
                    line=dict(color='navy', width=lw, dash='dash'),
                    name='Random Classification')
trace1 = go.Scatter(x=fpr_pca['LogisticRegression']['micro'], y=tpr_pca['LogisticRegression']['micro'], 
                    mode='lines', 
                    line=dict(color='aquamarine',width=lw),
                    name= 'Logistic Regression (area = %0.2f)' % auc(fpr_pca['LogisticRegression']['micro'], tpr_pca['LogisticRegression']['micro']))
trace2 = go.Scatter(x=fpr_pca['ExtraTreesClassifier']['micro'], y=tpr_pca['ExtraTreesClassifier']['micro'], 
                    mode='lines', 
                    line=dict(color='crimson',width=lw),
                    name= 'Extra Trees Classifier (area = %0.2f)' % auc(fpr_pca['ExtraTreesClassifier']['micro'], tpr_pca['ExtraTreesClassifier']['micro']))
trace3 = go.Scatter(x=fpr_pca['RandomForestClassifier']['micro'], y=tpr_pca['RandomForestClassifier']['micro'], 
                    mode='lines', 
                    line=dict(color='darkseagreen',width=lw),
                    name= 'Random Forest Classifier (area = %0.2f)' % auc(fpr_pca['RandomForestClassifier']['micro'], tpr_pca['RandomForestClassifier']['micro']))
trace4 = go.Scatter(x=fpr_pca['SVC']['micro'], y=tpr_pca['SVC']['micro'], 
                    mode='lines', 
                    line=dict(color='violet',width=lw),
                    name= 'SVC (area = %0.2f)' % auc(fpr_pca['SVC']['micro'], tpr_pca['SVC']['micro']))
trace5 = go.Scatter(x=fpr_pca['MLP']['micro'], y=tpr_pca['MLP']['micro'], 
                    mode='lines', 
                    line=dict(color='wheat',width=lw),
                    name= 'Multi-Layer Perceptron (area = %0.2f)' % auc(fpr_pca['MLP']['micro'], tpr_pca['MLP']['micro']))

data = [trace_mid,trace1,trace2,trace3,trace4,trace5]

layout = go.Layout(title="All Model ROC curve",
                   xaxis=dict(title='False Positive Rate'),
                   yaxis=dict(title='True Positive Rate'))
fig_roc_all = go.Figure(data=data, layout=layout)
py.iplot(fig_roc_all)

In [ ]:
# 7 estimators
roc_micros=[logistic_roc_data[-1],extratrees_roc_data[-1],
              rf_roc_data[-1],adaboost_roc_data[-1],\
              gd_roc_data[-1],svc_roc_data[-1],\
              mlp_roc_data[-1]]
layout = go.Layout(title="All Model ROC curve",
                   xaxis=dict(title='False Positive Rate'),
                   yaxis=dict(title='True Positive Rate'))
fig_roc_all = go.Figure(data=roc_micros, layout=layout)
py.iplot(fig_roc_all)

In [89]:
# 5 estimators
accuracy = go.Bar(
    x=['LogisticRegression', 'ExtraTrees', 'RandomForest','SVC','MLP'],
    y=[scores_pca['LogisticRegression'], scores_pca['ExtraTreesClassifier'], scores_pca['RandomForestClassifier'],\
       scores_pca['SVC'],scores_pca['MLP']],
    name='Accuracy'
)
precision = go.Bar(
    x=['LogisticRegression', 'ExtraTrees', 'RandomForest','SVC','MLP'],
    y=[report_pca['LogisticRegression'][2335:2339],report_pca['ExtraTreesClassifier'][2335:2339],\
      report_pca['RandomForestClassifier'][2335:2339],report_pca['SVC'][2335:2339],\
      report_pca['MLP'][2335:2339]],
    name='Precision'
)
recall = go.Bar(
    x=['LogisticRegression', 'ExtraTrees', 'RandomForest','SVC','MLP'],
    y=[report_pca['LogisticRegression'][2345:2345+4],report_pca['ExtraTreesClassifier'][2345:2345+4],\
      report_pca['RandomForestClassifier'][2345:2345+4],report_pca['SVC'][2345:2345+4],\
      report_pca['MLP'][2345:2345+4]],
    name='Recall'
)
f1 = go.Bar(
    x=['LogisticRegression', 'ExtraTrees', 'RandomForest','SVC','MLP'],
    y=[report_pca['LogisticRegression'][2355:2355+4],report_pca['ExtraTreesClassifier'][2355:2355+4],\
      report_pca['RandomForestClassifier'][2355:2355+4],report_pca['SVC'][2355:2355+4],\
      report_pca['MLP'][2355:2355+4]],
    name='F1-Score'
)
data = [accuracy, precision, recall,f1]
layout = go.Layout(
    barmode='group',
    title="Evaluation Metric Comparison of Various Models"
)

bar_comp = go.Figure(data=data, layout=layout)
py.iplot(bar_comp)

In [ ]:
## 7 estimators
accuracy = go.Bar(
    x=['LogisticRegression', 'ExtraTrees', 'RandomForest','AdaBoost','GradientBoost','SVC','MLP'],
    y=[scores_pca['LogisticRegression'], scores_pca['ExtraTreesClassifier'], scores_pca['RandomForestClassifier'],\
       scores_pca['AdaBoostClassifier'],scores_pca['GradientBoostingClassifier'],scores_pca['SVC'],scores_pca['MLP']],
    name='Accuracy'
)
precision = go.Bar(
    x=['LogisticRegression', 'ExtraTrees', 'RandomForest','AdaBoost','GradientBoost','SVC','MLP'],
    y=[report_pca['LogisticRegression']['micro avg']['precision'], 18, 29],
    name='Precision'
)
recall = go.Bar(
    x=['LogisticRegression', 'ExtraTrees', 'RandomForest','AdaBoost','GradientBoost','SVC','MLP'],
    y=[12, 18, 29],
    name='Recall'
)
f1 = go.Bar(
    x=['LogisticRegression', 'ExtraTrees', 'RandomForest','AdaBoost','GradientBoost','SVC','MLP'],
    y=[12, 18, 29],
    name='F1-Score'
)
data = [accuracy, precision, recall,f1]
layout = go.Layout(
    barmode='group',
    title="Evaluation Metric Comparison of Various Models"
)

bar_comp = go.Figure(data=data, layout=layout)
py.iplot(bar_comp)

In [ ]:
for key, value in report_pca.items() :
    print ("Classification Report: "+key)
    print (value +"\n")

# Data Visualization
## PCA 3D Scatter Plot

In [31]:
# PCA 3-component scatter plot
# Define Graph layout
layout = go.Layout(
    margin=dict(
        l=0,
        r=0,
        b=0,
        t=0
        )
)
X_train_pca3, X_test_pca3 = pcaSelection(X_train, X_test, 3)
pca3_traces = scatter3D(X_train_pca3)
fig1 = go.Figure(data=pca3_traces, layout=layout)
py.iplot(fig1, filename='PCA_3D_Scatter')

[2018-10-22 18:53:21,782 - GDC - INFO] X_train size after PCA: (8040, 3)
[2018-10-22 18:53:21,797 - GDC - INFO] X_test size after PCA: (3446, 3)
[2018-10-22 18:53:21,813 - GDC - INFO] Cumulative explained variation for 3 principal components: 0.1437


## t-SNE 3D Scatter Plot 

In [32]:
# tSNE 3-component scatter plot (PCA->tSNE)
X_train_tsne3 = tsneSelection(X_train_pca,3,1)
tsne3_traces = scatter3D(X_train_tsne3)
fig3 = go.Figure(data=tsne3_traces, layout=layout)
py.iplot(fig3, filename='tSNE_3D_Scatter')

[t-SNE] Computing 91 nearest neighbors...
[t-SNE] Indexed 8040 samples in 0.021s...
[t-SNE] Computed neighbors for 8040 samples in 9.581s...
[t-SNE] Computed conditional probabilities for sample 1000 / 8040
[t-SNE] Computed conditional probabilities for sample 2000 / 8040
[t-SNE] Computed conditional probabilities for sample 3000 / 8040
[t-SNE] Computed conditional probabilities for sample 4000 / 8040
[t-SNE] Computed conditional probabilities for sample 5000 / 8040
[t-SNE] Computed conditional probabilities for sample 6000 / 8040
[t-SNE] Computed conditional probabilities for sample 7000 / 8040
[t-SNE] Computed conditional probabilities for sample 8000 / 8040
[t-SNE] Computed conditional probabilities for sample 8040 / 8040
[t-SNE] Mean sigma: 2.000566
[t-SNE] KL divergence after 250 iterations with early exaggeration: 79.576187


[2018-10-22 19:02:13,016 - GDC - INFO] X_train size after tSNE: (8040, 3)


[t-SNE] KL divergence after 1000 iterations: 1.229386


## PCA 2D Scatter Plot

In [30]:
# PCA 2-component scatter plot
X_train_pca2, X_test_pca2 = pcaSelection(X_train, X_test, 2)
pca2_traces = scatter2D(X_train_pca2)
fig2 = go.Figure(data=pca2_traces, layout=layout)
py.iplot(fig2, filename='PCA_2D_Scatter')

[2018-10-22 18:52:51,701 - GDC - INFO] X_train size after PCA: (8040, 2)
[2018-10-22 18:52:51,704 - GDC - INFO] X_test size after PCA: (3446, 2)
[2018-10-22 18:52:51,705 - GDC - INFO] Cumulative explained variation for 2 principal components: 0.1153


## t-SNE 2D Scatter Plot

In [33]:
# tSNE 2-component scatter plot (PCA->tSNE)
X_train_tsne2 = tsneSelection(X_train_pca,2,1)
tsne2_traces = scatter2D(X_train_tsne2)
fig4 = go.Figure(data=tsne2_traces, layout=layout)
py.iplot(fig4, filename='tSNE_2D_Scatter')

[t-SNE] Computing 91 nearest neighbors...
[t-SNE] Indexed 8040 samples in 0.027s...
[t-SNE] Computed neighbors for 8040 samples in 11.423s...
[t-SNE] Computed conditional probabilities for sample 1000 / 8040
[t-SNE] Computed conditional probabilities for sample 2000 / 8040
[t-SNE] Computed conditional probabilities for sample 3000 / 8040
[t-SNE] Computed conditional probabilities for sample 4000 / 8040
[t-SNE] Computed conditional probabilities for sample 5000 / 8040
[t-SNE] Computed conditional probabilities for sample 6000 / 8040
[t-SNE] Computed conditional probabilities for sample 7000 / 8040
[t-SNE] Computed conditional probabilities for sample 8000 / 8040
[t-SNE] Computed conditional probabilities for sample 8040 / 8040
[t-SNE] Mean sigma: 2.000566
[t-SNE] KL divergence after 250 iterations with early exaggeration: 79.516594


[2018-10-22 19:04:29,807 - GDC - INFO] X_train size after tSNE: (8040, 2)


[t-SNE] KL divergence after 1000 iterations: 1.419539


# Evalutation
**Accuracy** - the set of labels predicted for a sample must exactly match the corresponding set of labels in y_true
**Precision** - tp / (tp + fp)
**Recall** - tp / (tp + fn)
**F1-score** -  2 * (precision * recall) / (precision + recall)
**Support** -  number of samples of the true response that lie in that class
**Confusion Matrix**- 
**ROC curve** - a graphical plot which illustrates the performance of a binary classifier system as its discrimination threshold is varied

# Conclusion